# Interactive Lyot coronagraph

M Kenworthy // Leiden Observatory // kenworthy@strw.leidenuniv.nl

Based on E. Por hcipy tutorials


The next cell is used in Google Colab to install `hcipy` - skip it if you already have it installed

In [17]:
%pip install hcipy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from hcipy import *
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches


import ipywidgets as widgets
from ipywidgets import interactive



In [10]:
pupil_grid = make_pupil_grid(128, 1.5)
focal_grid = make_focal_grid(8, 12)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

aperture = evaluate_supersampled(make_circular_aperture(1), pupil_grid, 4)


def addtext(a,s):
    a.text(0.05, 0.95, s, ha='left', va='top', transform=a.transAxes, color='white', fontsize=20)

def f(dy, mdim, b):
    fig, (x2, x3, x4, x5, x6) = plt.subplots(1,5,figsize=(14,4))
    
    for a in (x2,x3,x4,x5,x6):
        a.set_aspect('equal')

    for a in (x3,x4,x5,x6):
        a.get_yaxis().set_ticks([])
    
    wf = Wavefront(aperture * np.exp(2j * np.pi * pupil_grid.y * dy))

    img_ref = prop(wf).intensity
    
    focal_plane_mask_diameter = mdim
    focal_plane_mask_offset = [0, 0]

    focal_plane_mask_grid = make_focal_grid(64, focal_plane_mask_diameter / 2).shifted(focal_plane_mask_offset)

    focal_plane_mask = lambda grid: 1 - make_circular_aperture(focal_plane_mask_diameter, center=focal_plane_mask_offset)(grid)

    focal_plane_mask_evaluated = evaluate_supersampled(focal_plane_mask, focal_plane_mask_grid, 8)

    lyot_stop = evaluate_supersampled(make_circular_aperture(b), pupil_grid, 4)

    
    # make two coronagraph propagations to pupil plane - one with lyot mask, one without
    coro = LyotCoronagraph(pupil_grid, focal_plane_mask_evaluated, lyot_stop)
    coro_without_lyot = LyotCoronagraph(pupil_grid, focal_plane_mask_evaluated, None)

    
    img_before_fpm = prop(wf)

    imshow_field(np.log10(img_before_fpm.intensity / img_ref.max()), vmin=-5, vmax=0, cmap='inferno', ax=x2)
    addtext(x2,"B")

    img_after_fpm = img_before_fpm.copy()
    img_after_fpm.electric_field *=focal_plane_mask(img_after_fpm.electric_field.grid)

    C = img_after_fpm.intensity
    C[(C<1e-5)] = 1e-5
    
    print(C.min())
    
    imshow_field(np.log10(C / img_ref.max()), vmin=-5, vmax=0, cmap='inferno', ax=x3)
    addtext(x3,"C")

    img_before_lyot_stop = coro_without_lyot(wf)

    imshow_field(img_before_lyot_stop.intensity, vmax=1, cmap='inferno', ax=x4)
    addtext(x4,"D")

    img_after_lyot_stop = coro(wf)
    
    imshow_field(img_after_lyot_stop.intensity, vmax=1, cmap='inferno', ax=x5)
    addtext(x5,"E")

    img_science_plane = prop(coro(wf))
    imshow_field(np.log10(img_science_plane.intensity/img_ref.max()), vmin=-5, vmax=0, cmap='inferno', ax=x6)
    addtext(x6,"F")

    x2.scatter(0,0,marker='.',color='white')
    #x4.scatter(0,0,marker='.',color='white')
    x5.add_patch(patches.Circle((0,0), radius=0.5, fill=None,edgecolor='white',linestyle='--'))
    plt.show()

interactive_plot = interactive(f, dy=(0.0, 5.0), mdim=(0.5,4.0), b=(0.2, 1.5))
output = interactive_plot.children[-1]
output.layout.height = '210px'
interactive_plot

![Coronagraph layout](coronagraph.jpg)